# Example code for QPARC Challenge 2022

In [ ]:
# Import some modules.
# You are not allowed to import qulacs.QuantumState, nor other quantum circuit simulators.

from typing import Counter

import numpy as np
from openfermion.transforms import jordan_wigner
from qulacs import QuantumCircuit
from qulacs.gate import CZ, CNOT, RY, RX, RZ, H, X, Sdag
from scipy.optimize import minimize
from math import pi, floor
from noisyopt import minimizeSPSA
import itertools
from pennylane import qchem

from qparc import (
    QulacsExecutor,
    create_observable_from_openfermion_text,
    TotalShotsExceeded, 
)

In [ ]:
# Set up the executor, and get the problem hamiltonian.
# One must run quantum circuits always through the executor.
executor = QulacsExecutor()
fermionic_hamiltonian, n_qubits = executor.get_problem_hamiltonian()


# Process the Hamiltonian.
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)
qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

In [ ]:
pauli_coef, pauli_target, pauli_gate, pauli_str = get_terms_and_measurement_circuits(qulacs_hamiltonian)

opt_pauli_str, opt_pauli_coef = meas_optimization(pauli_str, pauli_coef) ##

print('# of terms before optimization:', len(pauli_coef))
print("# of terms after optimization:", len(opt_pauli_coef))

In [ ]:
# One can obtain HF energy and FCI energy.
print("HF energy:", executor.hf_energy)
print("FCI energy:", executor.fci_energy)

In [ ]:
# Set up the ansatz


def ry_ansatz_circuit(n_qubits, depth, theta_list):
    """ry_ansatz_circuit
    Returns Ry ansatz circuit.

    Args:
        n_qubits:
            The number of qubit used
        depth:
            Depth of the circuit.
        theta_list:
            Rotation angles.
    Returns:
        circuit:
            Resulting Ry ansatz circuit.
    """
    circuit = QuantumCircuit(n_qubits)
    params_id = 0
    for d in range(depth):
        for i in range(n_qubits):
            circuit.add_gate(
                RY(i, theta_list[params_id]),
            )
            params_id += 1
        for i in range(n_qubits // 2):
            circuit.add_gate(CZ(2 * i, 2 * i + 1))
        for i in range(n_qubits // 2 - 1):
            circuit.add_gate(CZ(2 * i + 1, 2 * i + 2))
    for i in range(n_qubits):
        circuit.add_gate(RY(i, theta_list[params_id]))
        params_id += 1

    return circuit

In [ ]:
def UCCSD_ansatz_circuit(n_qubits,depth, theta_list):
    
    circuit = QuantumCircuit(n_qubits)
    params_id = 0
    singles, doubles = qchem.excitations(n_qubits//2, n_qubits)
    
    
    for d in range(depth):
        
        # Single Excitation term
    
        for i, j in singles: ## 02, 03, 12, 13, 46, 47, 56, 57
            circuit.add_gate( RY(i, -pi/2))
            circuit.add_gate( RX(j, pi/2))
                
            for interval in range(i, j):
                circuit.add_gate( CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(j, theta_list[params_id]))
                
            for interval in range(j-1, i-1, -1):
                circuit.add_gate( CNOT(interval, interval+1))
                    
            circuit.add_gate( RY(i, pi/2))
            circuit.add_gate( RX(j, -pi/2))

            circuit.add_gate( RX(i, pi/2))
            circuit.add_gate( RY(j, -pi/2))
                
            for interval in range(i, j):
                circuit.add_gate( CNOT(interval, interval+1))
                    
            circuit.add_gate( RZ(j, -theta_list[params_id]))
                
            for interval in range(j-1, i-1, -1):
                circuit.add_gate( CNOT(interval, interval+1))
                    
            circuit.add_gate( RX(i, -pi/2))
            circuit.add_gate( RY(j, pi/2))
            params_id += 1    

        
            

        # Double Excitation term
        

        for i,j,p,q in doubles:
        
            ## 1st Round
            circuit.add_gate( H(i))
            circuit.add_gate( H(j))
            circuit.add_gate( RX(p,-pi/2))
            circuit.add_gate( H(q))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( H(i))
            circuit.add_gate( H(j))
            circuit.add_gate( RX(p,pi/2))
            circuit.add_gate( H(q))

            ## 2nd Round
            circuit.add_gate( RX(i, -pi/2))
            circuit.add_gate( H(j))
            circuit.add_gate( RX(p,-pi/2))
            circuit.add_gate( RX(q,-pi/2))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RX(i, pi/2))
            circuit.add_gate( H(j))
            circuit.add_gate( RX(p,pi/2))
            circuit.add_gate( RX(q,pi/2))
            
            ## 3rd Round
            circuit.add_gate( H(i))
            circuit.add_gate( RX(j,-pi/2))
            circuit.add_gate( RX(p,-pi/2))
            circuit.add_gate( RX(q,-pi/2))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( H(i))
            circuit.add_gate( RX(j,pi/2))
            circuit.add_gate( RX(p,pi/2))
            circuit.add_gate( RX(q,pi/2))
            
            ## 4th Round
            circuit.add_gate( H(i))
            circuit.add_gate( H(j))
            circuit.add_gate( H(p))
            circuit.add_gate( RX(q,-pi/2))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( H(i))
            circuit.add_gate( H(j))
            circuit.add_gate( H(p))
            circuit.add_gate( RX(q,pi/2))
            
            ## 5th Round
            circuit.add_gate( RX(i,-pi/2))
            circuit.add_gate( H(j))
            circuit.add_gate( H(p))
            circuit.add_gate( H(q))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, -theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RX(i,pi/2))
            circuit.add_gate( H(j))
            circuit.add_gate( H(p))
            circuit.add_gate( H(q))

            ## 6th Round
            circuit.add_gate( H(i))
            circuit.add_gate( RX(j,-pi/2))
            circuit.add_gate( H(p))
            circuit.add_gate( H(q))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, -theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( H(i))
            circuit.add_gate( RX(j,pi/2))
            circuit.add_gate( H(p))
            circuit.add_gate( H(q))
            
            ## 7th Round
            circuit.add_gate( RX(i,-pi/2))
            circuit.add_gate( RX(j,-pi/2))
            circuit.add_gate( RX(p,-pi/2))
            circuit.add_gate( H(q))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, -theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RX(i,pi/2))
            circuit.add_gate( RX(j,pi/2))
            circuit.add_gate( RX(p,pi/2))
            circuit.add_gate( H(q))
            
            ## 8th Round
            circuit.add_gate( RX(i,-pi/2))
            circuit.add_gate( RX(j,-pi/2))
            circuit.add_gate( H(p))
            circuit.add_gate( RX(q,-pi/2))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, -theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RX(i,pi/2))
            circuit.add_gate( RX(j,pi/2))
            circuit.add_gate( H(p))
            circuit.add_gate( RX(q,pi/2))
            
            params_id+=1

    return circuit
    
    

In [ ]:
def KUPCCGSD_ansatz_circuit(n_qubits, depth, theta_list):

    circuit = QuantumCircuit(n_qubits)
    params_id = 0

    def get_generalized_single(n_qubits):
    
        if n_qubits%2 !=0:
            ValueError("Please make sure n_qubits is even`1")

        alpha_orbitals = list(range(0,n_qubits,2))
        beta_orbitals = list(range(1,n_qubits,2))
        alpha_ex = itertools.combinations(alpha_orbitals,2)
        beta_ex = itertools.combinations(beta_orbitals,2)

        single_list = list(alpha_ex)+list(beta_ex)

        return single_list
    
    def get_paired_double(n_qubits):
        n_orb = n_qubits//2
        double_list = []

        orbital = []

        for i in range(n_orb):
            orbital.append((2*i,2*i+1))
        
        orbital_list = itertools.combinations(orbital,2)
        orbital_list = list(orbital_list)

        for ele in orbital_list:
            double_list.append(ele[0]+ele[1])
        
        return double_list



    for d in range(depth):
        ## Single Excitation
        for i,j in get_generalized_single(n_qubits):
                circuit.add_gate( RY(i, -pi/2))
                circuit.add_gate( RX(j, pi/2))
                    
                for interval in range(i, j):
                    circuit.add_gate( CNOT(interval, interval+1))
                    
                circuit.add_gate( RZ(j, theta_list[params_id]))
                    
                for interval in range(j-1, i-1, -1):
                    circuit.add_gate( CNOT(interval, interval+1))
                        
                circuit.add_gate( RY(i, pi/2))
                circuit.add_gate( RX(j, -pi/2))

                circuit.add_gate( RX(i, pi/2))
                circuit.add_gate( RY(j, -pi/2))
                    
                for interval in range(i, j):
                    circuit.add_gate( CNOT(interval, interval+1))
                        
                circuit.add_gate( RZ(j, -theta_list[params_id]))
                    
                for interval in range(j-1, i-1, -1):
                    circuit.add_gate( CNOT(interval, interval+1))
                        
                circuit.add_gate( RX(i, -pi/2))
                circuit.add_gate( RY(j, pi/2))
                params_id += 1    

        ## paired Double Excitation
        for i,j,p,q in get_paired_double(n_qubits):
            
            ## 1st Round
            circuit.add_gate( H(i))
            circuit.add_gate( H(j))
            circuit.add_gate( RX(p,-pi/2))
            circuit.add_gate( H(q))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( H(i))
            circuit.add_gate( H(j))
            circuit.add_gate( RX(p,pi/2))
            circuit.add_gate( H(q))

            ## 2nd Round
            circuit.add_gate( RX(i, -pi/2))
            circuit.add_gate( H(j))
            circuit.add_gate( RX(p,-pi/2))
            circuit.add_gate( RX(q,-pi/2))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RX(i, pi/2))
            circuit.add_gate( H(j))
            circuit.add_gate( RX(p,pi/2))
            circuit.add_gate( RX(q,pi/2))
            
            ## 3rd Round
            circuit.add_gate( H(i))
            circuit.add_gate( RX(j,-pi/2))
            circuit.add_gate( RX(p,-pi/2))
            circuit.add_gate( RX(q,-pi/2))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( H(i))
            circuit.add_gate( RX(j,pi/2))
            circuit.add_gate( RX(p,pi/2))
            circuit.add_gate( RX(q,pi/2))
            
            ## 4th Round
            circuit.add_gate( H(i))
            circuit.add_gate( H(j))
            circuit.add_gate( H(p))
            circuit.add_gate( RX(q,-pi/2))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( H(i))
            circuit.add_gate( H(j))
            circuit.add_gate( H(p))
            circuit.add_gate( RX(q,pi/2))
            
            ## 5th Round
            circuit.add_gate( RX(i,-pi/2))
            circuit.add_gate( H(j))
            circuit.add_gate( H(p))
            circuit.add_gate( H(q))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, -theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RX(i,pi/2))
            circuit.add_gate( H(j))
            circuit.add_gate( H(p))
            circuit.add_gate( H(q))

            ## 6th Round
            circuit.add_gate( H(i))
            circuit.add_gate( RX(j,-pi/2))
            circuit.add_gate( H(p))
            circuit.add_gate( H(q))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, -theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( H(i))
            circuit.add_gate( RX(j,pi/2))
            circuit.add_gate( H(p))
            circuit.add_gate( H(q))
            
            ## 7th Round
            circuit.add_gate( RX(i,-pi/2))
            circuit.add_gate( RX(j,-pi/2))
            circuit.add_gate( RX(p,-pi/2))
            circuit.add_gate( H(q))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, -theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RX(i,pi/2))
            circuit.add_gate( RX(j,pi/2))
            circuit.add_gate( RX(p,pi/2))
            circuit.add_gate( H(q))
            
            ## 8th Round
            circuit.add_gate( RX(i,-pi/2))
            circuit.add_gate( RX(j,-pi/2))
            circuit.add_gate( H(p))
            circuit.add_gate( RX(q,-pi/2))
            
            for interval in range(i, j):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(p, q):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RZ(q, -theta_list[params_id]/8))
            
            for interval in range(q-1, p-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate(CNOT(j, p))
            
            for interval in range(j-1, i-1, -1):
                circuit.add_gate(CNOT(interval, interval+1))
                
            circuit.add_gate( RX(i,pi/2))
            circuit.add_gate( RX(j,pi/2))
            circuit.add_gate( H(p))
            circuit.add_gate( RX(q,pi/2))
            
            params_id+=1
            

    return circuit

In [ ]:
def get_terms_and_measurement_circuits(observable):
    """get_terms_and_measurement_circuits
    Returns basis-rotation circuits for measurement, along with the corresponding terms.

    Args:
        observable:
            The observable to be measured.
    Returns:
        pauli_coef:
            List of coefficients.
        pauli_target:
            List of targetted qubits.
        pauli_gate:
            List of circuits for basis-rotation.
    """
    pauli_coef = []
    pauli_target = []
    pauli_gate = []
    pauli_str = []
    pauli_str_tmp = []
    n_qubits = observable.get_qubit_count()
    for i_term in range(observable.get_term_count()):
        term = observable.get_term(i_term)
        pauli_coef.append(term.get_coef())
        target_list = term.get_index_list()
        pauli_target.append(target_list)
        id_list = term.get_pauli_id_list()
        circuit = QuantumCircuit(n_qubits)
        
        pauli_str_tmp = ['I','I','I','I','I','I','I','I']
        for target, id in zip(target_list, id_list):
            
            
            if id == 1:
                circuit.add_gate(H(target))
                pauli_str_tmp[target]='X'
            elif id == 2:
                circuit.add_gate(Sdag(target))
                circuit.add_gate(H(target))
                pauli_str_tmp[target]='Y'
            elif id == 3:
                pauli_str_tmp[target]='Z'
                pass
            else:
                raise Exception(f"Operator {target, id} not supported")
        pauli_gate.append(circuit)
        pauli_str.append(pauli_str_tmp)

    return pauli_coef, pauli_target, pauli_gate, pauli_str

In [ ]:
def meas_optimization(pauli_str,pauli_coef): ## optimizing the measurement operators
    
    ## remove the identity operator 

    
    def check_simplification(op1, op2):
        
        for i in range(len(op1)):
            if op1[i]!=op2[i]:
                if op1[i]!="I" and op2[i]!="I":
                    return False      
        return True
            
            
    
    
    def join_operators(op1, op2):
        
        new_list = []
        for i in range(len(op1)):
            temp_element = op1[i]
            if op2[i]!="I":
                temp_element = op2[i]
            new_list.append(temp_element)
        return new_list
    
    opt_pauli_str = []
    opt_pauli_coef = []
    
    for op1, coef in zip(pauli_str,pauli_coef):
        added = False
        for i, op2 in enumerate(opt_pauli_str):
            
            if check_simplification(op1, op2):
                opt_pauli_str[i] = join_operators(op1, op2)
                opt_pauli_coef[i] += abs(coef)
                added = True
                break
        
        if not added and op1!=['I','I','I','I','I','I','I','I']:
            opt_pauli_str.append(op1)
            opt_pauli_coef.append(abs(coef))
            
            
            
    return opt_pauli_str, opt_pauli_coef

In [ ]:
def get_circuit(pauli_str):
    
    ## Don't input Identity to this Function!!!!
    n_qubits = len(pauli_str[0])
    circuit_list = []
    for op in pauli_str:
        circuit = QuantumCircuit(n_qubits)
        
        for i, term in enumerate(op):
            if term=='X':
                circuit.add_gate(H(i))
            elif term=='Y':
                circuit.add_gate(Sdag(i))
                circuit.add_gate(H(i))
            elif term=='Z':
                pass
            elif term=='I':
                pass
        
        circuit_list.append(circuit)
        
    return circuit_list


def find_suboperator(op, pauli_str, pauli_coef):
    
    ## Remove the identity operator first!!
    def check_simplification(op1, op2):
        for i in range(len(op1)):
            if op1[i]!=op2[i]:
                if op1[i]!="I" and op2[i]!="I":
                    return False      
        return True

    sub_op = []
    sub_coef = []
    for element, coef in zip(pauli_str, pauli_coef):
        if check_simplification(op, element):
            sub_op.append(element)
            sub_coef.append(coef)
            
    
    return sub_op, sub_coef
            
        
            

In [ ]:
def get_energy(theta_list, n_shots, depth, state, hamiltonian):
    """get_energy
    Returns the evaluated energy

    Args:
        theta_list:
            The parameters
        n_shots:
            The number of shots used to evaluate each term.
        depth:
            The depth of the ansatz
        state:
            The integer that defines the initial state in the computational basis.
        hamiltonian:
            The Hamiltonian to be evaluated.
    Returns:
        ret:
            The evaluated energy.
    """
    pauli_coef, pauli_target, pauli_gate, pauli_str = get_terms_and_measurement_circuits(
        hamiltonian
    )
    n_qubits = hamiltonian.get_qubit_count()
    #circuit = ry_ansatz_circuit(n_qubits=n_qubits, depth=depth, theta_list=theta_list)
    circuit = UCCSD_ansatz_circuit(depth=depth, theta_list=theta_list)
    
    total_shots = n_shots*len(pauli_coef)
    coef_tot = 0
    for coef in pauli_coef:
        coef_tot += abs(coef)
        
    ret = 0
    for coef, target, gate in zip(pauli_coef, pauli_target, pauli_gate):
        n_shots = floor(total_shots*abs(coef)/coef_tot)
        if target:
            counts = Counter(
                executor.sampling(
                    [circuit, gate],
                    state_int=state,
                    n_qubits=n_qubits,
                    n_shots=n_shots,
                )
            )
            for sample, count in counts.items():
                binary = np.binary_repr(sample).rjust(n_qubits, "0")
                measurement = np.product(
                    [-1 if binary[n_qubits - t - 1] == "1" else 1 for t in target]
                )
                ret += coef * measurement * count / n_shots
        else:
            ret += coef

    return ret.real

In [ ]:
def get_energy_ws(theta_list, n_shots,depth, state, hamiltonian): #Weighted Sampling
    """get_energy
    Returns the evaluated energy

    Args:
        theta_list:
            The parameters
        n_shots:
            The number of shots used to evaluate each term.
        depth:
            The depth of the ansatz
        state:
            The integer that defines the initial state in the computational basis.
        hamiltonian:
            The Hamiltonian to be evaluated.
    Returns:
        ret:
            The evaluated energy.
    """

    pauli_coef, pauli_target, pauli_gate, pauli_str = get_terms_and_measurement_circuits(hamiltonian)

    opt_pauli_str, opt_pauli_coef = meas_optimization(pauli_str, pauli_coef) ##
    opt_pauli_gate = get_circuit(opt_pauli_str)

    n_qubits = hamiltonian.get_qubit_count()
    #circuit = ry_ansatz_circuit(n_qubits=n_qubits, depth=depth, theta_list=theta_list)
    circuit = UCCSD_ansatz_circuit(n_qubits=n_qubits,depth=depth, theta_list=theta_list)
    #circuit = KUPCCGSD_ansatz_circuit(n_qubits=n_qubits, depth=depth, theta_list=theta_list)
    ret = 0

    coef_tot = 0
    for coef in opt_pauli_coef:
        coef_tot += abs(coef)

    total_shots = n_shots*len(opt_pauli_str) # Total shots for each cycle of sampling
    pauli_str_counter = pauli_str.copy()

    for shot_coef, gate, operator in zip(opt_pauli_coef, opt_pauli_gate, opt_pauli_str):

            n_shots = floor(total_shots*abs(shot_coef)/coef_tot)
            counts = Counter(
                executor.sampling(
                    [circuit, gate],
                    state_int=initial_state,
                    n_qubits=n_qubits,
                    n_shots= n_shots,
                )
            )
            #print(counts)
            subop_list, subop_coef = find_suboperator(operator, pauli_str, pauli_coef)
            #print("subop_list:", subop_list)
            #print("---------------------------")
            for subop, coef in zip(subop_list, subop_coef):

                #print("subop:", subop)
                if subop in pauli_str_counter:
                    target = [i for i, ele in enumerate(subop) if ele!="I"]

                else:
                    continue

                if target:
                    for sample, count in counts.items():
                        binary = np.binary_repr(sample).rjust(n_qubits, "0")
                        measurement = np.product(
                            [-1 if binary[n_qubits - t - 1] == "1" else 1 for t in target]
                        )
                        ret += coef * measurement * count / n_shots
                        #print("meas is: ", measurement)
                        #print("count ratio: ", count/ n_shots)
                else:
                    ret +=coef
                #print("# of term:", tiktok)
                #print("coef is:", coef)
                #print("subop is:", subop)
                #print("ret is now:", ret)       
                #print("---------------------------")
                pauli_str_counter.remove(subop)
                #tiktok +=1

    return ret.real

In [ ]:
# not used in the example
def get_gradient(theta_list, n_shots, depth, state, hamiltonian):
    """get_gradient
    Returns the evaluated gradient of the energy in the parameter space.

    Args:
        theta_list:
            The parameters
        n_shots:
            The number of shots used to evaluate each term.
        depth:
            The depth of the ansatz
        state:
            The integer that defines the initial state in the computational basis.
        hamiltonian:
            The Hamiltonian to be evaluated.
    Returns:
        np.array(g):
            The gradient of the energy in the parameter space.
    """
    g = []

    param_dim = len(theta_list)
    for i in range(param_dim):
        shift = np.zeros(param_dim)
        shift[i] = 0.5 * np.pi
        gi = 0.5 * (
            get_energy_ws(
                theta_list=theta_list + shift,
                n_shots=n_shots,
                depth=depth,
                state=state,
                hamiltonian=hamiltonian,
            )
            - get_energy_ws(
                theta_list=theta_list - shift,
                n_shots=n_shots,
                depth=depth,
                state=state,
                hamiltonian=hamiltonian,
            )
        )
        g.append(gi)
    return np.array(g)

In [ ]:
# Define input settings
n_shots = 6000
initial_state = 0b00001111
depth = 1

In [ ]:
# Define functions to be used in the optimization process.
def cost(theta_list, state=initial_state):
    ret = get_energy_ws(
        theta_list=theta_list,
        n_shots=n_shots,
        depth=depth,
        state=state,
        hamiltonian=qulacs_hamiltonian,
    )

    # executor.current_value will be used as the final result when the number of shots reach the limit,
    # so set the current value as often as possible, if you find a better energy.
    if ret < executor.current_value:
        executor.current_value = ret
    return ret


def grad(theta_list):
    ret = get_gradient(
        theta_list=theta_list,
        n_shots=n_shots,
        depth=depth,
        state=initial_state,
        hamiltonian=qulacs_hamiltonian,
    )
    return ret


def callback(theta_list):
    print("current val", executor.current_value)

In [54]:
# This block runs the VQE algorithm.
# The result will be finalized when executor.record_result() is called.

theta_length = 26 * depth ## for UCCSD ansatz
#theta_length = 18 * depth ## kupuccgsd ansatz
#init_theta_list = np.random.random(theta_length) * 0.02
init_theta_list = np.zeros(theta_length)
#method = "Powell"
method = "L-BFGS-B"
#method = "SLSQP"
options = {"disp": True, "maxiter": 10000}
try:
    opt = minimize(
        cost,
        init_theta_list,
        method=method,
        tol = 1e-4,
        jac=grad,
        options=options,
        callback=callback,
    )
except TotalShotsExceeded as e:
    print(e)
    executor.record_result()
else:
    print("Optimization finished without reaching the shot limit")
    executor.record_result()

current val -2.1499446843109853
The number of total shots exceeded the limit.

############## Result ##############
Resulting energy = -2.1499446843109853
total_shots = 99991053
------------------------------------
FCI energy = -2.1663874486347625
HF energy = -2.0985459369977617
####################################



In [ ]:
# This block runs the VQE algorithm.
# The result will be finalized when executor.record_result() is called.

theta_length = 26 * depth
init_theta_list = np.random.random(theta_length) * 0.01
#method = "Powell"
#method = "L-BFGS-B"

try:
    opt = minimizeSPSA(
        cost,
        x0=init_theta_list.copy(),
        niter=100,
        paired=False,
        c=0.1,
        a=0.3,
        callback=callback,
    )
except TotalShotsExceeded as e:
    print(e)
    executor.record_result()
else:
    print("Optimization finished without reaching the shot limit")
    executor.record_result()

In [ ]:
# This block is for final evaluation.
# The average accuracy will be evaluated.

executor.reset()
method = "L-BFGS-B"
options = {"disp": True, "maxiter": 10000}
for _ in range(10):
    init_theta_list = np.random.random(theta_length) * 0.01
    try:
        opt = minimize(
            cost,
            init_theta_list,
            method=method,
            jac=grad,
            options=options,
            callback=callback,
        )
    except TotalShotsExceeded as e:
        print(e)
        executor.record_result(verbose=False)
    else:
        print("Optimization finished without reaching the shot limit")
        executor.record_result(verbose=False)
executor.evaluate_final_result()